In [12]:
import pandas as pd
import requests

# API URL
api_url = "https://api.eia.gov/v2/electricity/operating-generator-capacity/data/"
api_key = "Kek68ciZeynS6IPayU8d4ePB2rOmuBQZJCYZbdx5"  # Replace with your actual API key

# API parameters
params = {
    'api_key': api_key,
    'frequency': 'monthly',
    'data[0]': 'latitude',
    'data[1]': 'longitude',
    'data[2]': 'net-summer-capacity-mw',
    'sort[0][column]': 'period',
    'sort[0][direction]': 'asc',
    'offset': 0,
    'length': 5000
}

# API request
response = requests.get(api_url, params=params)

# Check the status code of the response
if response.status_code == 200:
    data = response.json()
    records = data['response']['data']

    # Convert the data to a DataFrame
    df = pd.DataFrame(records)

    # Convert 'longitude' and 'latitude' to numeric, coercing errors to NaN
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

    # Modify the longitude values to be negative where necessary
    df['longitude'] = df['longitude'].apply(lambda x: -abs(x) if x > 0 else x)

    print(df.tail())  # Display the last few rows of the DataFrame

    # Check if 'energy_source_code' is available, if not, you may need to modify the request
    if 'energy_source_code' not in df.columns:
        print("The 'energy_source_code' column is not present in the data.")
    else:
        # Sort the DataFrame by 'energy_source_code' in descending order
        df_sorted = df.sort_values(by='energy_source_code', ascending=False)

        # Group by 'energy_source_code'
        grouped = df_sorted.groupby('energy_source_code')

        # Create an Excel writer
        with pd.ExcelWriter('EIA_Generator_Capacity_Data.xlsx', engine='openpyxl') as writer:
            for energy_source, group in grouped:
                group.to_excel(writer, sheet_name=str(energy_source), index=False)
                
        print("Data exported successfully to EIA_Generator_Capacity_Data.xlsx")
else:
    print(f"Failed to retrieve data: {response.status_code}")

       period stateid  stateName            sector        sectorName entityid  \
4995  2008-01      FL    Florida  electric-utility  Electric Utility     6616   
4996  2008-01      MI   Michigan  electric-utility  Electric Utility     1366   
4997  2008-01      MI   Michigan  electric-utility  Electric Utility     1366   
4998  2008-01      MI   Michigan  electric-utility  Electric Utility     1366   
4999  2008-01      WI  Wisconsin  electric-utility  Electric Utility     1278   

                      entityName plantid             plantName generatorid  \
4995  Fort Pierce Utilities Auth     658          Henry D King           8   
4996            City of Bay City    7398  Water Street Station        GEN1   
4997            City of Bay City    7399         Henry Station        GEN4   
4998            City of Bay City    7399         Henry Station        GEN3   
4999              City of Barron    4102                Barron          H2   

      ... prime_mover_code balancing_authori

In [33]:
import pandas as pd
import requests
import folium

# API URL
api_url = "https://api.eia.gov/v2/electricity/operating-generator-capacity/data/"
api_key = "Kek68ciZeynS6IPayU8d4ePB2rOmuBQZJCYZbdx5"  

# API parameters
params = {
    'api_key': api_key,
    'frequency': 'monthly',
    'data[0]': 'latitude',
    'data[1]': 'longitude',
    'data[2]': 'net-summer-capacity-mw',
    'sort[0][column]': 'period',
    'sort[0][direction]': 'asc',
    'offset': 0,
    'length': 5000
}

# API request
response = requests.get(api_url, params=params)

# Check the status code of the response
if response.status_code == 200:
    data = response.json()

    # Convert the data to a DataFrame
    df = pd.DataFrame(data['response']['data'])

    # Convert 'longitude' and 'latitude' to numeric
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

    # Modify the longitude values to be negative
    df['longitude'] = df['longitude'].apply(lambda x: -abs(x) if x > 0 else x)

    # Define your grouping logic
    grouping_logic = {
        'BIT': 'Coal', 'BLQ': 'Coal', 'LIG': 'Coal', 'PC': 'Coal', 'RC': 'Coal',
        'RFO': 'Coal', 'SUB': 'Coal', 'WC': 'Coal',
        'NG': 'Natural Gas', 'BFG': 'Natural Gas', 'OG': 'Natural Gas', 'LFG': 'Natural Gas', 'PG': 'Natural Gas',
        'DFO': 'Oil', 'JF': 'Oil', 'KER': 'Oil', 'PUR': 'Oil', 'WO': 'Oil',
        'GEO': 'Geothermal', 'H2': 'Geothermal',
        'SUN': 'Solar', 'WND': 'Wind', 'WAT': 'Hydro',
        'AB': 'Renewable', 'OBG': 'Renewable', 'OBL': 'Renewable', 'OBS': 'Renewable',
        'SLW': 'Renewable', 'WDL': 'Renewable', 'WDS': 'Renewable',
        'NUC': 'Nuclear', 'OTH': 'Other'
    }

    # Add a new column for your custom groups based on the energy source
    df['custom_group'] = df['energy_source_code'].map(grouping_logic)

    # Group the data by 'custom_group'
    df_grouped = df.groupby(['custom_group', 'plantid'], as_index=False).agg({
        'net-summer-capacity-mw': 'sum',
        'latitude': 'first',
        'longitude': 'first',
        'entityName': 'first',
        'plantName': 'first',
        'stateName': 'first'
    })

    # Create a base map
    base_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

    # Create a feature group for each custom group
    for group_name in df_grouped['custom_group'].unique():
        feature_group = folium.FeatureGroup(name=group_name)
        base_map.add_child(feature_group)

        # Filter the DataFrame for the current group
        group_df = df_grouped[df_grouped['custom_group'] == group_name]

        # Add markers to the feature group
        for index, row in group_df.iterrows():
            if pd.notna(row['latitude']) and pd.notna(row['longitude']):
                folium.Marker(
                    location=[row['latitude'], row['longitude']],
                    popup=(
                        f"<table>"
                        f"<tr><th>Plant Name</th><td>{row['plantName']}</td></tr>"
                        f"<tr><th>Utility</th><td>{row['entityName']}</td></tr>"
                        f"<tr><th>State</th><td>{row['stateName']}</td></tr>"
                        f"<tr><th>Capacity</th><td>{row['net-summer-capacity-mw']} MW</td></tr>"
                        f"<tr><th>Plant ID</th><td>{row['plantid']}</td></tr>"
                        f"</table>"
                    ),
                    tooltip=row['plantName'],
                    icon=folium.Icon(color=color_map.get(group_name, 'gray'), icon='info-sign')
                ).add_to(feature_group)

    # Add LayerControl to toggle the layers
    folium.LayerControl(collapsed=False).add_to(base_map)

    # Save the map with all layers
    map_filename = "all_energy_types_locations_map.html"
    base_map.save(map_filename)
    print(f"Map with grouped energy types saved as {map_filename}")

else:
    print(f"Failed to retrieve data: {response.status_code}")

Map with grouped energy types saved as all_energy_types_locations_map.html


In [27]:
import requests
import pandas as pd
import folium

# Path to the Excel file
excel_file_path = 'C:/Users/PALMME/Python stuff/Main project/EIA_Generator_Capacity_Data.xlsx'

# Define a color map for different energy sources
color_map = {
    'Coal': 'black',
    'Natural Gas': 'blue',
    'Nuclear': 'red',
    'Hydro': 'green',
    'Wind': 'purple',
    'Solar': 'orange',
    'Renewable': 'lightgreen',
    'Oil': 'darkred',
    'Other': 'gray'
}

# Define your grouping logic
grouping_logic = {
    'BIT': 'Coal', 'BLQ': 'Coal', 'LIG': 'Coal', 'PC': 'Coal', 'RC': 'Coal',
    'RFO': 'Coal', 'SUB': 'Coal', 'WC': 'Coal',
    'NG': 'Natural Gas', 'BFG': 'Natural Gas', 'OG': 'Natural Gas', 'LFG': 'Natural Gas', 'PG': 'Natural Gas',
    'DFO': 'Oil', 'JF': 'Oil', 'KER': 'Oil', 'PUR': 'Oil', 'WO': 'Oil',
    'GEO': 'Geothermal', 'H2': 'Geothermal',
    'SUN': 'Solar', 'WND': 'Wind', 'WAT': 'Hydro',
    'AB': 'Renewable', 'OBG': 'Renewable', 'OBL': 'Renewable', 'OBS': 'Renewable',
    'SLW': 'Renewable', 'WDL': 'Renewable', 'WDS': 'Renewable',
    'NUC': 'Nuclear', 'OTH': 'Other'
}

# Load the Excel file
excel_data = pd.ExcelFile(excel_file_path)

# Create a base map
base_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Create a FeatureGroup for the legend
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 150px; height: 200px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:white;">
    &nbsp;<b>Legend</b><br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:black"></i>&nbsp;Coal<br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:blue"></i>&nbsp;Natural Gas<br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp;Nuclear<br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp;Hydro<br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:purple"></i>&nbsp;Wind<br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:orange"></i>&nbsp;Solar<br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:lightgreen"></i>&nbsp;Renewable<br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:darkred"></i>&nbsp;Oil<br>
    &nbsp;<i class="fa fa-circle fa-1x" style="color:gray"></i>&nbsp;Other<br>
    </div>
'''
base_map.get_root().html.add_child(folium.Element(legend_html))

# Initialize an empty DataFrame to consolidate data from all sheets
all_data = pd.DataFrame()

# Iterate through each sheet (energy type) in the Excel file
for sheet_name in excel_data.sheet_names:
    # Read the data from the sheet
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    # Drop rows with NaNs in latitude or longitude
    df = df.dropna(subset=['latitude', 'longitude'])

    # Convert 'longitude' and 'latitude' to numeric, coercing errors to NaN
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

    # Modify the longitude values to be negative
    df['longitude'] = df['longitude'].apply(lambda x: -abs(x) if x > 0 else x)

    # Consolidate data into a single DataFrame
    all_data = pd.concat([all_data, df], ignore_index=True)

# Check if 'energy_source_code' exists in the consolidated DataFrame
if 'energy_source_code' in all_data.columns:
    # Add a new column for your custom groups based on the energy source
    all_data['custom_group'] = all_data['energy_source_code'].map(grouping_logic)

    # Group the data by 'custom_group'
    df_grouped = all_data.groupby(['custom_group', 'plantid'], as_index=False).agg({
        'net-summer-capacity-mw': 'sum',
        'latitude': 'first',
        'longitude': 'first',
        'entityName': 'first',
        'plantName': 'first',
        'stateName': 'first'
    })

    # Create a feature group for each custom group
    for group_name in df_grouped['custom_group'].unique():
        feature_group = folium.FeatureGroup(name=group_name)
        base_map.add_child(feature_group)

        # Filter the DataFrame for the current group
        group_df = df_grouped[df_grouped['custom_group'] == group_name]

        # Add markers to the feature group
        for index, row in group_df.iterrows():
            if pd.notna(row['latitude']) and pd.notna(row['longitude']):
                folium.Marker(
                    location=[row['latitude'], row['longitude']],
                    popup=(
                        f"<table>"
                        f"<tr><th>Plant Name</th><td>{row['plantName']}</td></tr>"
                        f"<tr><th>Utility</th><td>{row['entityName']}</td></tr>"
                        f"<tr><th>State</th><td>{row['stateName']}</td></tr>"
                        f"<tr><th>Capacity</th><td>{row['net-summer-capacity-mw']} MW</td></tr>"
                        f"<tr><th>Plant ID</th><td>{row['plantid']}</td></tr>"
                        f"</table>"
                    ),
                    tooltip=row['plantName'],
                    icon=folium.Icon(color=color_map.get(group_name, 'gray'), icon='info-sign')
                ).add_to(feature_group)

# Add LayerControl to toggle the layers
folium.LayerControl(collapsed=False).add_to(base_map)

# Save the map with all layers
map_filename = "all_energy_types_locations_map.html"
base_map.save(map_filename)
print(f"Map with grouped energy types saved as {map_filename}")

Map with grouped energy types saved as all_energy_types_locations_map.html
